# 1. Imports

In [15]:
import pandas as pd

## Definições

In [16]:
colunas_redundantes = [
    'unidade_cadastro',
    'cod_regulacao',
    'cod_unidade_origem',
    'tipo_regulacao',
    'data_cadastro',
    'data_lista_reg',
    'data_vaga_reg',
    'data_admissao_reg',
    'data_lista_hospital',
    'data_vaga_hospital',
    'data_internacao',
    'data_cancel_solicitacao',
    'data_transferencia',
    'gravidez',
    'tipo_internacao_de',
    'data_expiracao',
    'cod_transf_alta',
    'nome_unidade_solicitacao',
    'nome_unidade_destino',
    'via_internacao',
    'idade',
    'faixa_etaria',
    'diff_in_sec',
    'diff_in_min',
    'diff_in_hour',
    'diff_in_day',
    'diff_delay_Rank_vaga_internacao_1',
    'Count',
    'total_repeat_cod',
    'total_repeat',
    'sum_los_hour',
    'sum_outside_hour',
    'cumsum',
    'cumsum.1',
    'cumsum.1(start)',
    'cod_paciente.1',
    'start',
    'stop',
    'uf',
    'municipio',
    'idade',
    'idade_aprox',
    'status_internacao',
    'tipo_alta',
    'tipo_internacao',
    'tempo_espera_internacao',
    'tempo_espera_aceite',
    'cid10_diagnostico_alta',
    'cid10_seg_diagnostico_alta',
    'grupo_diagnostico_alta',
    'tempo_internacao',
    'event',
    'cid10_seg_diagnostico',
    'data_solicitacao'
]

PATH_INPUT_DATASET = '../data/raw/total dataset with los and outsid time.csv'
PATH_MAPPING_PROFISSAO = "../data/raw/profissoes_mapping.txt"
PATH_OUTPUT_PROCESSED_DATASET = "../data/processed/"
PATH_DE_PARA_CID = '../data/raw/CID_DEPARA.csv'


estado_civil_mapping = {
    'SOLTEIRO': 'solteiro',
    'CASADO': 'casado',
    'NÃO INFORMADO': 'nao informado',
    'viasvo': 'nao informado',
    'SEPARADO': 'solteiro',
    'DIVORCIADO': 'solteiro'
}

etnia_mapping = {
    'BRANCO': 'branco',
    'PARDO': 'pardo',
    'NEGRO': 'preto',
    'NÃO INFORMADO': 'nao informado',
    'AMARELO': 'amarelo',
}

arranjo_domiciliar_mapping = {
    'NÃO INFORMADO': 'nao informado',
    'SOZINHO': 'sozinho',
    'COM COMPANHEIRO ( A)': 'acompanhado',
    'COM FILHOS(AS)': 'acompanhado',
    'COM PAI/MÃE': 'acompanhado',
    'COM OUTROS FAMILIARES': 'acompanhado',
    'OUTRO': 'nao informado',
    'CONHECIDOS/AMIGOS': 'acompanhado'
}

## Funções de apoio

In [17]:
def ler_dataset(path, **kwargs):
    
    return pd.read_csv(path, **kwargs)

def pegar_mapping_profissao(path):
    
    with open(path, "r") as arquivo:
        
        return eval(arquivo.read())

def remover_colunas_redundantes(df, cols_to_drop):

    return df.drop(cols_to_drop, axis=1)

def tratar_colunas_data(df):
    colunas_simples_tratamento = ['data_internacao_real', 'data_alta']
    colunas_tratamento_regex = ['data_nascimento']
    padrao_data_regex = "(\d{1,2}/\d{1,2}/\d{4})"

    for coluna in colunas_simples_tratamento:
        df[coluna] = pd.to_datetime(df[coluna], format='%m/%d/%Y %H:%M', errors='raise')

    for coluna in colunas_tratamento_regex:
        df[coluna] = pd.to_datetime(df[coluna].str.extract(padrao_data_regex).to_numpy().flat, format='%m/%d/%Y')

    return df

def criar_situacao_profissao(df):
    profissao_mapping = pegar_mapping_profissao(PATH_MAPPING_PROFISSAO)
    
    df['profissao_ocupacao'] = df['profissao_ocupacao'].str.lower()
    df['situacao_profissao'] = df['profissao_ocupacao'].map(lambda x: profissao_mapping.get(x, 'nao informado'))
    df = df.drop('profissao_ocupacao', axis=1)

    return df

def tratar_estado_civil(df):
    df['estado_civil'] = df['estado_civil'].map(estado_civil_mapping)

    return df

def tratar_etnia(df):
    df['etnia'] = df['etnia'].map(etnia_mapping)

    return df

def tratar_arranjo_domiciliar(df):
    df['arranjo_domiciliar'] = df['arranjo_domiciliar'].map(arranjo_domiciliar_mapping)

    return df

def tratar_cid_diagnostico(df):
    cid_df = ler_dataset(PATH_DE_PARA_CID, sep=';', encoding='latin-1')

    df['cid10_diagnostico'] = df['cid10_diagnostico'].str.replace('.', '', regex=False).str.upper()
    df = (
        df
        .merge(cid_df[['SUBCATEGORIA', 'GRUPO']], left_on='cid10_diagnostico', right_on='SUBCATEGORIA', how='left')
        .drop('SUBCATEGORIA', axis=1)
        .rename(columns={'GRUPO': 'cid_diagnostico_grupo_descr'})
        .fillna({'cid_diagnostico_grupo_descr': 'nao informado'})
    )

    cid_grupos_dict = {}
    cid_grupos_unicos = df['cid_diagnostico_grupo_descr'].sort_values().unique()
    for i, descricao in enumerate(cid_grupos_unicos):
        grp = f"CID_GRP_{str(i).zfill(2)}"
        cid_grupos_dict[descricao] = grp

    df['cid_diagnostico_grupo_chave'] = df['cid_diagnostico_grupo_descr'].map(cid_grupos_dict)

    return df

def tratar_colunas_binarias(df):
    cols_binarias = [
        'traumatismo',
        'prob_respiratorios',
        'avc',
        'convulsao',
        'has',
        'diabetes',
        'doenca_infecto'
    ]

    for col in cols_binarias:
        df[col] = df[col].apply(lambda x: 1 if x == 'SIM' else 0)

    return df

def remover_nulos(df):

    return df.dropna()

def extrair_features_de_data(df):

    df['ano_data_internacao'] = df['data_internacao_real'].dt.year
    df['mes_data_internacao'] = df['data_internacao_real'].dt.month
    df['idade_data_internacao'] = (df['data_internacao_real'] - df['data_nascimento']).dt.days // 365

    df['tempo_internacao_horas'] = (df['data_alta'] - df['data_internacao_real'])
    # Substituir valores negativos (datas de alta anteriores à data de internação) por 0 horas
    df.loc[df['tempo_internacao_horas'] < pd.Timedelta(0), 'tempo_internacao_horas'] = pd.Timedelta(0)
    df['tempo_internacao_horas'] = df['tempo_internacao_horas'].dt.total_seconds() / 3600

    return df

def criar_target(df):
    
    paciente_counts = df['cod_paciente'].value_counts()
    df['reinternacao'] = df['cod_paciente'].map(lambda x: 1 if paciente_counts[x] > 1 else 0)

    return df

def filtrar_primeira_internacao_por_paciente(df):
    
    df_sorted = df.sort_values(by='data_internacao_real')
    df_sorted['row_number'] = df_sorted.groupby('cod_paciente')['data_internacao_real'].rank(method='first')

    primeira_data_internacao = df_sorted[df_sorted['row_number'] == 1]

    return primeira_data_internacao

# 2. Dataprep

In [18]:
df = ler_dataset(PATH_INPUT_DATASET, encoding="latin1")

colunas_redundantes_pos_data_prep = [
    'data_internacao_real',
    'data_nascimento',
    'cod_paciente',
    'cid10_diagnostico',
    'data_alta',
    'cid_diagnostico_grupo_descr',
    'row_number'
]

transformed_df = (
    df
    .pipe(remover_colunas_redundantes, colunas_redundantes)
    .pipe(tratar_colunas_data)
    .pipe(criar_situacao_profissao)
    .pipe(tratar_estado_civil)
    .pipe(tratar_etnia)
    .pipe(tratar_arranjo_domiciliar)
    .pipe(tratar_cid_diagnostico)
    .pipe(tratar_colunas_binarias)
    .pipe(remover_nulos)
    .pipe(extrair_features_de_data)
    .pipe(criar_target)
    .pipe(filtrar_primeira_internacao_por_paciente)
    .pipe(remover_colunas_redundantes, colunas_redundantes_pos_data_prep)
)

transformed_df

,sexo,estado_civil,etnia,arranjo_domiciliar,traumatismo,prob_respiratorios,avc,convulsao,has,diabetes,doenca_infecto,situacao_profissao,cid_diagnostico_grupo_chave,ano_data_internacao,mes_data_internacao,idade_data_internacao,tempo_internacao_horas,reinternacao
260,F,solteiro,branco,sozinho,0,0,0,0,0,0,0,inativa,CID_GRP_25,2012,8,23,670.500000,1
403,M,nao informado,preto,nao informado,0,0,0,0,0,0,0,nao informado,CID_GRP_28,2012,8,22,24.266667,1
485,F,nao informado,nao informado,nao informado,0,0,0,0,0,0,0,ativa,CID_GRP_19,2012,8,32,2191.733333,0
368,F,solteiro,nao informado,nao informado,0,0,0,0,0,0,0,inativa,CID_GRP_25,2012,8,14,133.916667,0
508,F,casado,branco,acompanhado,0,0,0,0,0,0,0,nao informado,CID_GRP_23,2012,8,28,133.616667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8753,F,nao informado,branco,nao informado,0,0,0,0,1,1,0,inativa,CID_GRP_27,2017,12,66,589.400000,0
8749,F,solteiro,pardo,acompanhado,0,0,0,0,0,0,0,inativa,CID_GRP_25,2017,12,26,404.883333,0
8544,M,solteiro,pardo,acompanhado,0,0,0,0,0,0,0,nao informado,CID_GRP_28,2017,12,32,124.250000,0
8752,F,nao informado,nao informado,nao informado,0,0,0,0,0,0,0,nao informado,CID_GRP_23,2017,12,15,97.216667,0


In [19]:
transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4599 entries, 260 to 8754
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sexo                         4599 non-null   object 
 1   estado_civil                 4599 non-null   object 
 2   etnia                        4599 non-null   object 
 3   arranjo_domiciliar           4599 non-null   object 
 4   traumatismo                  4599 non-null   int64  
 5   prob_respiratorios           4599 non-null   int64  
 6   avc                          4599 non-null   int64  
 7   convulsao                    4599 non-null   int64  
 8   has                          4599 non-null   int64  
 9   diabetes                     4599 non-null   int64  
 10  doenca_infecto               4599 non-null   int64  
 11  situacao_profissao           4599 non-null   object 
 12  cid_diagnostico_grupo_chave  4599 non-null   object 
 13  ano_data_interna

# Salvando dataset tratado

In [20]:
nome = 'internacoes_processado.csv'
path_to_save = PATH_OUTPUT_PROCESSED_DATASET + nome

transformed_df.to_csv(path_to_save, index=False)

In [21]:
new_df = pd.read_csv(path_to_save)
new_df

,sexo,estado_civil,etnia,arranjo_domiciliar,traumatismo,prob_respiratorios,avc,convulsao,has,diabetes,doenca_infecto,situacao_profissao,cid_diagnostico_grupo_chave,ano_data_internacao,mes_data_internacao,idade_data_internacao,tempo_internacao_horas,reinternacao
0,F,solteiro,branco,sozinho,0,0,0,0,0,0,0,inativa,CID_GRP_25,2012,8,23,670.500000,1
1,M,nao informado,preto,nao informado,0,0,0,0,0,0,0,nao informado,CID_GRP_28,2012,8,22,24.266667,1
2,F,nao informado,nao informado,nao informado,0,0,0,0,0,0,0,ativa,CID_GRP_19,2012,8,32,2191.733333,0
3,F,solteiro,nao informado,nao informado,0,0,0,0,0,0,0,inativa,CID_GRP_25,2012,8,14,133.916667,0
4,F,casado,branco,acompanhado,0,0,0,0,0,0,0,nao informado,CID_GRP_23,2012,8,28,133.616667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594,F,nao informado,branco,nao informado,0,0,0,0,1,1,0,inativa,CID_GRP_27,2017,12,66,589.400000,0
4595,F,solteiro,pardo,acompanhado,0,0,0,0,0,0,0,inativa,CID_GRP_25,2017,12,26,404.883333,0
4596,M,solteiro,pardo,acompanhado,0,0,0,0,0,0,0,nao informado,CID_GRP_28,2017,12,32,124.250000,0
4597,F,nao informado,nao informado,nao informado,0,0,0,0,0,0,0,nao informado,CID_GRP_23,2017,12,15,97.216667,0
